## Laptop Sales Prediction: Data Preparation

In [1]:
# Import libraries for data processing
import numpy as np 
import pandas as pd 

# Import libraries for data visualisation
import matplotlib.pyplot as plt 

In [2]:
# Expand the pandas dataframe
pd.set_option("display.max_columns", 100)

Data Prepartion: Approach

+ Data Preprocessing
+ Data Transformation
+ Feature Engineering (p1)

In [3]:
# Load the dataset
ebay_laptop_file = "ebay_laptop_dataset.csv"
laptop_df = pd.read_csv(ebay_laptop_file)
laptop_df.head(3)

,Brand,Price,Currency,Color,Features,Condition,Condition Description,Seller Note,GPU,Processor,Processor Speed,Processor Speed Unit,Type,Width of the Display,Height of the Display,OS,Storage Type,Hard Drive Capacity,Hard Drive Capacity Unit,SSD Capacity,SSD Capacity Unit,Screen Size (inch),Ram Size,Ram Size Unit
0,other,303.8,$,gray,"Backlit Keyboard, Built-in Microphone, Built...",New,"A brand-new, unused, unopened, undamaged item ...",undefined,intel,quad core,3.80,GHz,notebook/laptop,2160.0,1440.0,windows,ssd,512.0,gb,1.0,tb,14,8.0,gb
1,dell,400.0,$,black,"Backlit Keyboard, Bluetooth, Built-in Micropho...",Very Good - Refurbished,The item shows minimal wear and is backed by a...,aaa pcs is a microsoft authorized refurbisher ...,intel,intel core i7 8th generation,4.20,GHz,notebook/laptop,1920.0,1080.0,windows,ssd,2.0,tb,NaN,unknown,14,NaN,unknown
2,dell,175.0,$,black,"10/100 LAN Card, Backlit Keyboard, Bluetooth, ...",Used,An item that has been used previously. The ite...,"well kept, fully functional, includes battery,...",intel,intel core i5-6300u,2.40,GHz,notebook/laptop,1920.0,1080.0,windows,ssd,500.0,gb,500.0,gb,14,16.0,gb


## Data Preprocessing

+ Removing null values missing values
+ Detecting and Removing inconsistencies
+ Removing irrelevant data

In [4]:
# Show the number of features within the dataset
num_features = laptop_df.shape[1]
print(f"Features: {laptop_df.columns}")
print(f"Number of features: {num_features}")

Features: Index(['Brand', 'Price', 'Currency', 'Color', 'Features', 'Condition',
       'Condition Description', 'Seller Note', 'GPU', 'Processor',
       'Processor Speed', 'Processor Speed Unit', 'Type',
       'Width of the Display', 'Height of the Display', 'OS', 'Storage Type',
       'Hard Drive Capacity', 'Hard Drive Capacity Unit', 'SSD Capacity',
       'SSD Capacity Unit', 'Screen Size (inch)', 'Ram Size', 'Ram Size Unit'],
      dtype='object')
Number of features: 24


In [5]:
# Identify the datatypes of the features
feature_datatypes = laptop_df.dtypes.to_frame("DataTypes")
missing_values = laptop_df.isnull().sum().to_frame("MissingValues")

# Combine the frames --> feature dataframe
feature_df = pd.concat([feature_datatypes, missing_values], axis=1).reset_index()
feature_df = feature_df.rename(columns={"index": "Features"})
feature_df

# Filter the features with missing values
feature_miss = feature_df[feature_df["MissingValues"] > 0]
feature_miss_df = laptop_df[feature_miss["Features"].values]
feature_miss_df
# print(f"Features with missing values: {feature_miss.shape[0]}")

,Features,Processor Speed,Width of the Display,Height of the Display,Hard Drive Capacity,SSD Capacity,Screen Size (inch),Ram Size
0,"Backlit Keyboard, Built-in Microphone, Built...",3.80,2160.0,1440.0,512.0,1.0,14,8.0
1,"Backlit Keyboard, Bluetooth, Built-in Micropho...",4.20,1920.0,1080.0,2.0,NaN,14,NaN
2,"10/100 LAN Card, Backlit Keyboard, Bluetooth, ...",2.40,1920.0,1080.0,500.0,500.0,14,16.0
3,"Bluetooth, Built-in Microphone, Built-in Webca...",2.40,1366.0,768.0,16.0,NaN,11.6,4.0
4,"10/100 LAN Card, Built-in Microphone, Built-in...",1.40,1366.0,768.0,NaN,256.0,12.5,8.0
...,...,...,...,...,...,...,...,...
4178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4180,"Touchscreen, 10/100 LAN Card, Bluetooth, Ba...",2.80,1920.0,1080.0,NaN,120.0,12.5,NaN
4181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**1. Inconsistent Values**

In [6]:
# Detect inconsistencies
for feature in laptop_df.columns[:3]:
    # Check if all values are convertible to numeric values
    is_convertible = "Convertible" if pd.api.types.is_numeric_dtype(laptop_df[feature]) else "Not Convertible"
    print(f"{feature}: {is_convertible}")

# Form a dataframe where features are convertible to numeric data
def is_convertible(feature ):
    return "YES" if pd.api.types.is_numeric_dtype(laptop_df[feature]) else "NO"

feature_df["Convertible"] = feature_df["Features"].transform(is_convertible)

# Filter features that are convertible and non textual
features_nonconv = feature_df[(feature_df["Convertible"] == "NO" )]["Features"].values
inconsistent_features = laptop_df[features_nonconv][["Processor Speed", "Screen Size (inch)"]]
inconsistent_features.head()

Brand: Not Convertible
Price: Convertible
Currency: Not Convertible


,Processor Speed,Screen Size (inch)
0,3.80,14
1,4.20,14
2,2.40,14
3,2.40,11.6
4,1.40,12.5


In [7]:
# Create sequences from Processor Speed and Screen Size (inch)
processor_speed = laptop_df["Processor Speed"].values
screen_size = laptop_df["Screen Size (inch)"].values 

# Detect inconsistent data in the deataset
def detect_inconsistencies(sequences: list[str]) -> list:
    # Define lists to store real and inconsistent values 
    incon_list = []
    real_list = []
    
    for value in sequences: 
        try:
            # check with special values or characters are inside 
            real_value = float(value)
            real_list.append(real_value)
        except ValueError:
            incon_list.append(value)

    # Return inconsistent list
    return incon_list

incon_lst1 = detect_inconsistencies(processor_speed)
incon_lst2 = detect_inconsistencies(screen_size)

print(f"Processor speed: {incon_lst1}")
print(f"Screen Size: {incon_lst2}")

Processor speed: ['o4.2', 'o3.4', 'o2.8', 'o3.9', 'o4.5', 'o3.3', 'o3.6', 'o3.4', 'o2.6', 'o3.4', '@1.7', 'o4.3', 'o3.4', 'o4.4']
Screen Size: ['14.1.', '12.3.', '14.1.', '12()', "1314'", '12.5(', '11.6.', '15.6.', '12.5/', '15.4.', '15.6.', '11.6.', '14.1.', '13.3.', '14.1.', '14.1.', '15.6.', '14.1.', '.', '10.1(', '11.6.', '12.5.', '13.3+', '14-15']


In [8]:
# Remove inconsistent data based on give info
laptop_df["Processor Speed"] = laptop_df["Processor Speed"].str.extract(r"(\d+\.\d+)")
laptop_df["Screen Size (inch)"] = laptop_df["Screen Size (inch)"].str.extract(r"(\d.+\.\d+|\d+)")

# Convert the feature to float
laptop_df["Processor Speed"] = laptop_df["Processor Speed"].astype(float)
laptop_df["Screen Size (inch)"] = laptop_df["Screen Size (inch)"].astype(float)

**2. Missing Values**

In [9]:
# Remove the nullvalues from numerical features
laptop_df["Processor Speed"] = laptop_df["Processor Speed"].fillna(0)
laptop_df["Width of the Display"] = laptop_df["Width of the Display"].fillna(0)
laptop_df["Height of the Display"] = laptop_df["Height of the Display"].fillna(0)
laptop_df["Hard Drive Capacity"] = laptop_df["Hard Drive Capacity"].fillna(0)
laptop_df["SSD Capacity"] = laptop_df["SSD Capacity"].fillna(0)
laptop_df["Screen Size (inch)"] = laptop_df["Screen Size (inch)"].fillna(0)
laptop_df["Ram Size"] = laptop_df["Ram Size"].fillna(0)

# Remove the nullvalues of categorical features 
laptop_df["Features"] = laptop_df["Features"].fillna("No specifications")

In [10]:
# Compute the mean value to replace all zero values
compute_mean = lambda feature: np.mean(laptop_df[feature].values)

laptop_df["Processor Speed"] = laptop_df["Processor Speed"].replace(0.0, compute_mean("Processor Speed"))
laptop_df["Width of the Display"] = laptop_df["Width of the Display"].replace(0.0, compute_mean("Width of the Display"))
laptop_df["Height of the Display"] = laptop_df["Height of the Display"].replace(0.0, compute_mean("Height of the Display"))
laptop_df["Hard Drive Capacity"] = laptop_df["Hard Drive Capacity"].replace(0.0, compute_mean("Hard Drive Capacity"))
laptop_df["SSD Capacity"] = laptop_df["SSD Capacity"].replace(0.0, compute_mean("SSD Capacity"))
laptop_df["Screen Size (inch)"] = laptop_df["Screen Size (inch)"].replace(0.0, compute_mean("Screen Size (inch)"))
laptop_df["Ram Size"] = laptop_df["Ram Size"].replace(0.0, compute_mean("Ram Size"))

In [11]:
laptop_df.head()

,Brand,Price,Currency,Color,Features,Condition,Condition Description,Seller Note,GPU,Processor,Processor Speed,Processor Speed Unit,Type,Width of the Display,Height of the Display,OS,Storage Type,Hard Drive Capacity,Hard Drive Capacity Unit,SSD Capacity,SSD Capacity Unit,Screen Size (inch),Ram Size,Ram Size Unit
0,other,303.80,$,gray,"Backlit Keyboard, Built-in Microphone, Built...",New,"A brand-new, unused, unopened, undamaged item ...",undefined,intel,quad core,3.8,GHz,notebook/laptop,2160.0,1440.0,windows,ssd,512.000000,gb,1.000000,tb,14.0,8.000000,gb
1,dell,400.00,$,black,"Backlit Keyboard, Bluetooth, Built-in Micropho...",Very Good - Refurbished,The item shows minimal wear and is backed by a...,aaa pcs is a microsoft authorized refurbisher ...,intel,intel core i7 8th generation,4.2,GHz,notebook/laptop,1920.0,1080.0,windows,ssd,2.000000,tb,136.448482,unknown,14.0,8.089649,unknown
2,dell,175.00,$,black,"10/100 LAN Card, Backlit Keyboard, Bluetooth, ...",Used,An item that has been used previously. The ite...,"well kept, fully functional, includes battery,...",intel,intel core i5-6300u,2.4,GHz,notebook/laptop,1920.0,1080.0,windows,ssd,500.000000,gb,500.000000,gb,14.0,16.000000,gb
3,hp,85.00,$,black,"Bluetooth, Built-in Microphone, Built-in Webca...",Good - Refurbished,The item shows moderate wear and is backed by ...,1-year allstate warranty. the original hp char...,intel,intel celeron n,2.4,GHz,notebook/laptop,1366.0,768.0,chrome,emmc,16.000000,gb,136.448482,unknown,11.6,4.000000,gb
4,dell,101.25,$,other,"10/100 LAN Card, Built-in Microphone, Built-in...",Good - Refurbished,The item shows moderate wear and is backed by ...,laptops is tested & fully working with some si...,other,intel core i5 6th generation,1.4,GHz,notebook/laptop,1366.0,768.0,windows,ssd,59.964619,unknown,256.000000,gb,12.5,8.000000,gb


In [12]:
laptop_df.select_dtypes([int, float]).columns

Index(['Price', 'Processor Speed', 'Width of the Display',
       'Height of the Display', 'Hard Drive Capacity', 'SSD Capacity',
       'Screen Size (inch)', 'Ram Size'],
      dtype='object')

In [13]:
# Modify the format of the real values
laptop_df["Processor Speed"] = laptop_df["Processor Speed"].apply(lambda x: round(x, 3))
laptop_df["Width of the Display"] = laptop_df["Width of the Display"].apply(lambda x: round(x, 3))
laptop_df["Height of the Display"] = laptop_df["Height of the Display"].apply(lambda x: round(x, 3))
laptop_df["Hard Drive Capacity"] = laptop_df["Hard Drive Capacity"].apply(lambda x: round(x, 3))
laptop_df["SSD Capacity"] = laptop_df["SSD Capacity"].apply(lambda x: round(x, 3))
laptop_df["Screen Size (inch)"] = laptop_df["Screen Size (inch)"].apply(lambda x: round(x, 3))
laptop_df["Ram Size"] = laptop_df["Ram Size"].apply(lambda x: round(x, 3))

# Show the dataset with new formatted values
laptop_df.select_dtypes([int, float])

,Price,Processor Speed,Width of the Display,Height of the Display,Hard Drive Capacity,SSD Capacity,Screen Size (inch),Ram Size
0,303.80,3.800,2160.000,1440.000,512.000,1.000,14.000,8.00
1,400.00,4.200,1920.000,1080.000,2.000,136.448,14.000,8.09
2,175.00,2.400,1920.000,1080.000,500.000,500.000,14.000,16.00
3,85.00,2.400,1366.000,768.000,16.000,136.448,11.600,4.00
4,101.25,1.400,1366.000,768.000,59.965,256.000,12.500,8.00
...,...,...,...,...,...,...,...,...
4178,162.20,1.188,779.242,447.949,59.965,136.448,9.077,8.09
4179,93.25,1.188,779.242,447.949,59.965,136.448,9.077,8.09
4180,424.80,2.800,1920.000,1080.000,59.965,120.000,12.500,8.09
4181,90.94,1.188,779.242,447.949,59.965,136.448,9.077,8.09


In [14]:
# Check nullvalues after removal 
laptop_df.isnull().sum()

Brand                       0
Price                       0
Currency                    0
Color                       0
Features                    0
Condition                   0
Condition Description       0
Seller Note                 0
GPU                         0
Processor                   0
Processor Speed             0
Processor Speed Unit        0
Type                        0
Width of the Display        0
Height of the Display       0
OS                          0
Storage Type                0
Hard Drive Capacity         0
Hard Drive Capacity Unit    0
SSD Capacity                0
SSD Capacity Unit           0
Screen Size (inch)          0
Ram Size                    0
Ram Size Unit               0
dtype: int64

**3. Irrelevant Data**

In [15]:
laptop_df.head()

,Brand,Price,Currency,Color,Features,Condition,Condition Description,Seller Note,GPU,Processor,Processor Speed,Processor Speed Unit,Type,Width of the Display,Height of the Display,OS,Storage Type,Hard Drive Capacity,Hard Drive Capacity Unit,SSD Capacity,SSD Capacity Unit,Screen Size (inch),Ram Size,Ram Size Unit
0,other,303.80,$,gray,"Backlit Keyboard, Built-in Microphone, Built...",New,"A brand-new, unused, unopened, undamaged item ...",undefined,intel,quad core,3.8,GHz,notebook/laptop,2160.0,1440.0,windows,ssd,512.000,gb,1.000,tb,14.0,8.00,gb
1,dell,400.00,$,black,"Backlit Keyboard, Bluetooth, Built-in Micropho...",Very Good - Refurbished,The item shows minimal wear and is backed by a...,aaa pcs is a microsoft authorized refurbisher ...,intel,intel core i7 8th generation,4.2,GHz,notebook/laptop,1920.0,1080.0,windows,ssd,2.000,tb,136.448,unknown,14.0,8.09,unknown
2,dell,175.00,$,black,"10/100 LAN Card, Backlit Keyboard, Bluetooth, ...",Used,An item that has been used previously. The ite...,"well kept, fully functional, includes battery,...",intel,intel core i5-6300u,2.4,GHz,notebook/laptop,1920.0,1080.0,windows,ssd,500.000,gb,500.000,gb,14.0,16.00,gb
3,hp,85.00,$,black,"Bluetooth, Built-in Microphone, Built-in Webca...",Good - Refurbished,The item shows moderate wear and is backed by ...,1-year allstate warranty. the original hp char...,intel,intel celeron n,2.4,GHz,notebook/laptop,1366.0,768.0,chrome,emmc,16.000,gb,136.448,unknown,11.6,4.00,gb
4,dell,101.25,$,other,"10/100 LAN Card, Built-in Microphone, Built-in...",Good - Refurbished,The item shows moderate wear and is backed by ...,laptops is tested & fully working with some si...,other,intel core i5 6th generation,1.4,GHz,notebook/laptop,1366.0,768.0,windows,ssd,59.965,unknown,256.000,gb,12.5,8.00,gb
